In [ ]:
import os
import json
import numpy as np
import torch
from torch import nn
import torchvision
import random
from tqdm import *
from PIL import Image
from io import StringIO, BytesIO
import lpips

import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader


from pro_gan_pytorch.utils import adjust_dynamic_range
from torch.nn.functional import interpolate
import torchvision.transforms.functional as fn
from train_log import MeanTracker

from torch.utils.tensorboard import SummaryWriter

import model_rs as recsys_models

In [ ]:
data_train = 'amazon'

if data_train == 'amazon':

    dataset_name = 'AmazonFashion6ImgPartitioned.npy'
    dataset = np.load('../DVBPR/dataset/'+ dataset_name, encoding='bytes', allow_pickle=True)
    [user_train, _, _, Item, usernum, itemnum] = dataset

elif data_train == 'tradesy':

    dataset_name = 'TradesyImgPartitioned.npy'
    dataset = np.load('../DVBPR/data/' + dataset_name, encoding='bytes')
    [user_train, user_validation, user_test, Item, usernum, itemnum] = dataset
    cold_list = np.load('../data/tradesy_one_k_cold.npy')

def default_loader(path):
    img_pil =  Image.open(BytesIO(path)).convert('RGB')
    img_tensor = input_transform(img_pil)
    return img_tensor


input_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])

class trainset(Dataset):
    def __init__(self, loader=default_loader):
        self.images_i = file_train_i
        self.images_j = file_train_j
        self.target = train_ls
        self.loader = loader

    def __getitem__(self, index):
        fn_i = self.images_i[index]
        img_i = self.loader(fn_i)
        fn_j = self.images_j[index]
        img_j = self.loader(fn_j)
        target = self.target[index]
        return img_i, img_j, target[0], target[1], target[2]

    def __len__(self):
        return len(self.images_i)

# helper scale function
def scale(x):
    # assume x is scaled to (0, 1)
    # scale to feature_range and return scaled x
    x = (x - x.min()) / (x.max() - x.min())
    return x

def scale_synth(x):
    # scale to feature_range and return scaled x
    x = adjust_dynamic_range(x,drange_in=(-1.0, 1.0), drange_out=(0.0, 1.0))
    return x

def scale_percept(x):
    # scale to feature_range and return scaled x
    x = adjust_dynamic_range(x,drange_in=(-1.0, 1.0), drange_out=(-1.0, 1.0))

    return x

def manual_normalize(x, mean, std):
    mean_ten = torch.Tensor(mean).unsqueeze(dim=0).unsqueeze(dim=2).unsqueeze(dim=3).cuda()
    std_ten = torch.Tensor(std).unsqueeze(dim=0).unsqueeze(dim=2).unsqueeze(dim=3).cuda()
    x = (x-mean_ten)/std_ten
    return x

def scale_rs(x):
    x = adjust_dynamic_range(x,drange_in=(-1.0, 1.0), drange_out=(0.0, 1.0))
    x = interpolate(x, size=(224, 224), mode='bilinear')
    x = manual_normalize(x, mean=[0.6949, 0.6748, 0.6676], std=[0.3102, 0.3220, 0.3252])
    return x

##### Initialize Dataset Object ######
def sample(user):
    u = random.randrange(usernum)
    numu = len(user[u])
    i = user[u][random.randrange(numu)][b'productid']
    M=set()
    for item in user[u]:
        M.add(item[b'productid'])
    while True:
        j=random.randrange(itemnum)
        if (not j in M): break
    return (u,i,j)

oneiteration = 0
for item in user_train: oneiteration+=len(user_train[item])

train_ls = [list(sample(user_train)) for _ in range(oneiteration)]

file_train_i = [Item[i][b'imgs'] for i in range(itemnum)]
file_train_j = [Item[i][b'imgs'] for i in range(itemnum)]

# class_val = 0
# file_train_i = [Item[i][b'imgs'] for i in range(itemnum) if class_val in np.argwhere(Item[i][b'c']==True)]
# file_train_j = [Item[i][b'imgs'] for i in range(itemnum) if class_val not in np.argwhere(Item[i][b'c']==True)]


train_dataset  = trainset()

In [ ]:
### Loading all the data points
# preference_scores = torch.load('raw_data_exp/preference_score_U_p_class2.pt') 
# base_preference_scores = torch.load('raw_data_exp/base_preference_score_U_p_class2.pt') 
# base_images = torch.load('raw_data_exp/base_images_U_p_class2.pt')
# shifted_images = torch.load('raw_data_exp/shifted_images_U_p_class2.pt')
# actual_images = torch.load('raw_data_exp/item_images_U_p_class2.pt')

preference_scores = torch.load('raw_data_exp/multi_user/preference_score_U_10000.pt') 
# base_preference_scores = torch.load('raw_data_exp/multi_user/base_preference_score_U_10000.pt') 
base_preference_scores = torch.load('raw_data_exp/multi_user/base_preference_score_U_10000.pt').unsqueeze(dim=1)
base_images = torch.load('raw_data_exp/multi_user/base_images_U_10000.pt')
shifted_images = torch.load('raw_data_exp/multi_user/shifted_images_U_10000.pt')
# actual_images = torch.load('raw_data_exp/multi_user/item_images_U_10000.pt')


In [ ]:
from torchmetrics import StructuralSimilarityIndexMeasure

ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

changed_based_images = []

for i in base_images:
    changed_based_images.append(i.unsqueeze(dim=0).repeat(6,1,1,1))

changed_based_images = torch.cat(changed_based_images, dim=0)

ssim_scores = []
for i in range(6000):
    ssim_score = ssim(changed_based_images[i].unsqueeze(dim=0), shifted_images[i].unsqueeze(dim=0))
    ssim_scores.append(1-ssim_score)

ssim_scores = torch.Tensor(ssim_scores)

print('opposite SSIM Score Mean: ', ssim_scores.mean())
print('opposite SSIM Score Std: ', ssim_scores.std())


In [ ]:
new_sorted_pref_score = []
average_over = 3
for i in base_preference_scores:
    i1 = i.sort(descending=True)[0][:average_over].unsqueeze(dim=0)
    new_sorted_pref_score.append(i1)

new_base_sorted_pref_score = torch.cat(new_sorted_pref_score, dim=0)

print('Mean Pref: ', new_base_sorted_pref_score.mean())
print('Std Pref: ', new_base_sorted_pref_score.std())
print('Var Pref: ', new_base_sorted_pref_score.var())

In [ ]:
new_sorted_pref_score = []
average_over = 3
for i in preference_scores:
    i1 = i.sort(descending=True)[0][:average_over].unsqueeze(dim=0)
    new_sorted_pref_score.append(i1)

new_sorted_pref_score = torch.cat(new_sorted_pref_score, dim=0)

print('Mean Pref: ', new_sorted_pref_score.mean())
print('Std Pref: ', new_sorted_pref_score.std())
print('Var Pref: ', new_sorted_pref_score.var())


In [ ]:
shifted_in_score = (new_sorted_pref_score - new_base_sorted_pref_score)
print('Mean Pref: ', shifted_in_score.mean())
print('Std Pref: ', shifted_in_score.std())
print('Var Pref: ', shifted_in_score.var())

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

# from torchvision.models.inception import inception_v3
from torchvision.models import inception_v3, Inception_V3_Weights

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(weights=Inception_V3_Weights.DEFAULT, transform_input=False).type(dtype)
    inception_model.eval()
    up = nn.Upsample(size=(290, 290), mode='bilinear').type(dtype)
    def get_pred(x):
        # print(x.shape)
        # print(x.max())
        # print(x.min())
        if resize:
            x = up(x)
        # print(x.shape)
        # print(x.max())
        # print(x.min())
        # a = c
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, [batch] in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [ ]:
shifted_images.shape

In [ ]:
from torch.utils.data import TensorDataset
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# img_dset = TensorDataset(normalize(scale_synth(batch_image1)))
k = torch.ones(1000,3,128,128)
img_dset = TensorDataset(shifted_images)

(is_mean, is_std) = inception_score(img_dset, cuda=False, batch_size=32, resize=True, splits=10)
print(('Mean Inception: {:.3f}').format(is_mean))
print(('Mean Inception: {:.3f}').format(is_std))

In [ ]:
data_loader_train = DataLoader(train_dataset, batch_size = 1000, shuffle=True, num_workers = 4, drop_last=True)

In [ ]:
batch_image1, _, _, _, _ = next(iter(data_loader_train))

In [ ]:
class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=False,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        
        inception = inception_v3(weights=Inception_V3_Weights.DEFAULT)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp
    
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])

In [ ]:
def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

In [ ]:
from scipy import linalg

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

In [ ]:
def calculate_fretchet(images_real,images_fake,model):
     mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=False)
     mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=False)
    
     """get fretched distance"""
     fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
     return fid_value

In [ ]:
calculate_fretchet(batch_image1,shifted_images,model)